# 15.6 针对序列级和词元级应用程序微调BERT
- **目录**
  - 15.6.1 BERT单文本分类
  - 15.6.2 BERT文本对分类或回归
  - 15.6.3 BERT文本标注
  - 15.6.4 BERT问答

- 本章前几节中为自然语言处理应用设计了不同的模型。
  - 例如基于循环神经网络、卷积神经网络、注意力和多层感知机。
- 这些模型在有空间或时间限制的情况下是有帮助的，
  - 但是，为每个自然语言处理任务精心设计一个**特定的模型**实际上是**不可行**的。
- 在 14.8节中，我们介绍了一个名为BERT的预训练模型，该模型可以对广泛的自然语言处理任务进行**最少的架构更改**。
  - 一方面，在提出时，BERT改进了各种自然语言处理任务的技术水平。
  - 另一方面，正如在 14.10节中指出的那样，原始BERT模型的两个版本分别带有1.1亿和3.4亿个参数。
- 因此，当有足够的计算资源时，我们可以考虑为下游自然语言处理应用**微调BERT**。
- 下面，我们将**自然语言处理应用的子集概括为序列级和词元级** 。
  * 在序列层次上，介绍了在单文本分类任务和文本对分类（或回归）任务中，如何将文本输入的BERT表示转换为输出标签。
  * 在词元级别，我们将简要介绍新的应用，如文本标注和问答，并说明BERT如何表示它们的输入并转换为输出标签。
  * 在微调期间，不同应用之间的BERT所需的“**最小架构更改**”是**额外的**全连接层。
  * **在下游应用的监督学习期间，额外层的参数是从零开始学习的，而预训练BERT模型中的所有参数都是微调的**。
 

- **要点：**
  - 设计多种NLP模型存在实际挑战。
  - BERT是一个适用于多个NLP任务的预训练模型。
  - 对于序列级和词元级任务，BERT只需最小架构更改。
  - BERT模型参数量大，需要充足的计算资源。
  - 微调BERT时，除新增层参数外，所有预训练参数都需微调。

---------------
- **说明：**
- **（1）何为"序列级"和"词元级"任务？**
  - "序列级"和"词元级"任务是自然语言处理中的两类基本任务，它们按照处理的粒度和任务的性质进行分类：
    - 序列级任务关注**整个文本序列**的意义和分类，而词元级任务则关注对文本中**具体词元**的识别和分类。
    - 在应用BERT或其他预训练模型进行微调时，这种区分帮助确定需要进行的**架构调整**，以及对应的**损失函数和输出层设计**。
  - **序列级任务**（Sequence-level tasks）关注的是整个文本序列的处理和理解。在这类任务中，模型的输出通常是关于整个输入文本的单一结果。这些任务的特点是结果不需要对每个单词或词元进行区别，而是得到一个整体的输出。序列级任务的例子包括：
    - **文本分类(Text Classification)**：判断整个文本的类别，如垃圾邮件检测、情感分析。
    - **自然语言推断(Natural Language Inference, NLI)**：给定一个前提和一个假设，判断假设是真实的，假的还是无法确定的。
    - **情感分析(Sentiment Analysis)**：判断文本所表达情感的倾向，例如正面、负面或中性。
  - **词元级任务**（Token-level tasks）则涉及对文本中的单个词元（如单词或字符）进行处理和理解。模型必须对文本中的每一个词元给出具体的输出或判断。词元级任务的例子包括：
    - **命名实体识别(Named Entity Recognition, NER)**：识别文本中的特定命名实体（如人名、地点、机构名称）。
    - **词性标注(Part-of-Speech Tagging, POS)**：为文本中的每个词分配适当的词性（如名词、动词、形容词等）。
    - **语义角色标注(Semantic Role Labeling, SRL)**：分析句子中各个词汇的语义角色，如谁是行动的执行者、目标对象等。
    - **问答(Question Answering, QA)**：在问答任务中识别回答问题所需的关键词或短语。

- **（2）如何理解：在微调期间，不同应用之间的BERT所需的“最小架构更改”是额外的全连接层？**
  - 当使用BERT模型来微调不同的下游自然语言处理任务时，通常不需要对BERT的核心架构进行大幅度的修改。
  - 相反，**只需要增加一个额外的全连接层（即一个线性层或密集层），以适应特定任务的输出**。
  - “最小架构更改”意味着：
    - **保持BERT核心**: 预训练的BERT模型核心，包括它的层和参数，是不变的。
      - 这些层已经通过大规模的数据集学习了语言的**通用特征和表示**。
    - **额外全连接层的角色**:
       - 对于**分类任务**，这个额外的全连接层通常会输出一个固定大小的向量，**其维度与分类任务的类别数量相匹配**。
       - 对于**回归任务**，这个层通常输出一个单一的数值，表示某个连续的目标变量，比如两个句子间的语义相似度。
    - **任务特异性训练**: 在微调过程中，虽然BERT的参数也会进行微调来更好地适应特定任务，但新增的全连接层参数是特定于任务的，会从零开始训练。这表明预训练BERT模型已经具备了很强的语言表示能力，通常只需少量的修改即可适用于不同的NLP任务。

- **（3）如何理解：在下游应用的监督学习期间，额外层的参数是从零开始学习的，而预训练BERT模型中的所有参数都是微调的？**
  - 这句话说明了在BERT模型用于具体NLP任务时，针对任务特定输出所添加的**新层是完全重新学习**的，而**预训练模型中的参数则是基于它们之前学习到的知识进行细微调整**的。
  - 这允许模型既能够利用预训练中获得的丰富语言知识，又能够适应特定任务的特殊需求。
  - 同时说明了在使用预训练的BERT模型对特定下游任务进行微调时的两个关键步骤：
     -  **额外层的参数学习**:
        - 当BERT模型被应用于一个具体的下游任务时，需要一个额外层（通常是一个或多个全连接层）来适配特定的任务需求，如分类标签的预测。
        - 这个额外层是为特定任务新加的，其参数在开始微调前没有被训练过，因此是从零开始学习的。
        - 也就是说，这些参数是随机初始化的，并会在下游任务的监督学习过程中训练和调整。
     - **预训练BERT模型参数的微调**:
       - 预训练的BERT模型已经通过大量的通用数据进行了训练，这使它学到了语言的各种特征和规律。
       - 在下游任务中，这些预训练的参数不会重新开始学习，而是从预训练时已经学到的知识状态开始，通过进一步的训练（微调）来适应具体任务。
       - 微调允许模型在保留预训练时学到的通用知识的同时，根据特定任务的数据进行调整，以更好地执行特定的下游任务。

- **（4）在微调过程中，BERT模型的参数是如何更新和调整的？**
  - 在微调过程中，BERT模型的参数更新和调整，遵循标准的深度学习训练过程，特别是使用梯度下降法。以下是使用微调BERT模型时的参数更新步骤：
    - 1. **初始化**:
       - 在开始微调之前，BERT模型的参数会被初始化为预训练阶段学到的权重。
    - 2. **数据准备**:
      - 微调所用的数据集需要根据具体任务进行预处理，如分类标签或者其他相关的任务特定信息。
    - 3. **添加额外层**:
      - 通常情况下，会在BERT模型的顶部增加一个或者几个全连接层（对于不同的任务可有所不同）以适应特定的输出要求。
    - 4. **前向传播**:
      - 在训练阶段，输入数据通过BERT模型进行前向传播，经过多层的变换最终到达额外添加的全连接层，得到输出结果。
    - 5. **计算损失**:
      - 根据任务的具体需求（例如分类、回归、序列标注等），使用一个损失函数来计算模型当前输出和真实标签之间的差距。
    - 6. **反向传播**:
      - 损失函数的结果用来进行反向传播。在这个过程中，通过链式法则计算模型每层参数相对于损失的梯度。
    - 7. **参数更新**:
      - 使用梯度和学习率来更新模型参数。通常使用优化算法如Adam，这种算法可以通过调整学习速率来改进梯度下降。
    - 8. **重复迭代**:
      - 将整个数据集分批次输入模型，并重复上述的训练步骤，每个批次更新一次模型的参数。
    - 9. **学习率调整**:
      - 有时在训练过程中会根据一定的策略调整学习率，如逐步衰减或使用预热阶段的学习率调整策略。
    - 10. **早停**:
      - 为了防止过拟合，通常会在验证集上跟踪模型的性能，如果性能在一定迭代后没有显著提升可能会提前结束训练。
  - 这个微调过程涉及所有参数的细微调整，以便学习从预训练阶段转移到特定任务的完成情况。
  - 最终随着训练的进行，模型逐渐从通用的语言理解调整到特定任务的需求。

--------------

## 15.6.1 BERT单文本分类
- **单文本分类**将单个文本序列作为输入，并输出其分类结果。
- 除了我们在这一章中探讨的情感分析之外，**语言可接受性语料库（Corpus of Linguistic Acceptability，COLA）** 也是一个单文本分类的数据集，它的要求判断给定的句子在**语法上是否可以接受**。
- 例如，“I should study.”是可以接受的，但是“I should studying.”不是可以接受的。
<center><img src='../img/bert-one-seq.svg'></center>
<center>图15.6.1 微调BERT用于单文本分类应用，如情感分析和测试语言可接受性（这里假设输入的单个文本有六个词元）</center><br>
- 14.8节描述了BERT的输入表示。
  - BERT输入序列明确地表示单个文本和文本对。
  - 其中特殊分类标记“&lt;cls&gt;”用于序列分类，而特殊分类标记“&lt;sep&gt;”标记单个文本的结束或分隔成对文本。
- 如图15.6.1所示，在单文本分类应用中，特殊分类标记“&lt;cls&gt;”的BERT表示对整个输入文本序列的信息进行编码。
- 作为输入单个文本的表示，它将被送入到由全连接（稠密）层组成的小型多层感知机中，以输出所有离散标签值的分布。

- **要点：**
  -  **单文本分类**: 输入是单个文本序列，输出是这段文本的分类。
  - **数据集举例**: 情感分析和语言可接受性语料库（COLA）检测句子的语法是否可以接受。
  - **句子示例**: "I should study."（可以接受）与"I should studying."（不可接受）。
  - **BERT输入**: 使用特殊标记"\<cls\>"表示序列的开始，用于分类任务；用"\<sep\>"标记文本结尾或分隔文本对。
  - **模型结构**: 在BERT的"\<cls\>"标记表示编码了整个文本序列的信息，这个信息随后被使用在一个全连接层上，来预测不同标签值的分布。

## 15.6.2 BERT文本对分类或回归
- 在本章中，我们还研究了自然语言推断。它属于**文本对分类**，这是一种对文本进行分类的应用类型。
- 以一对文本作为输入但输出连续值，**语义文本相似度**是一个流行的“文本对**回归**”任务。
- 这项任务评估句子的语义相似度。
  - 例如，在语义文本相似度基准数据集（Semantic Textual Similarity Benchmark）中，句子对的相似度得分是从0（无语义重叠）到5（语义等价）的分数区间。
- 我们的目标是预测这些分数。来自语义文本相似性基准数据集的样本包括（句子1，句子2，相似性得分）：
  * "A plane is taking off."（“一架飞机正在起飞。”），"An air plane is taking off."（“一架飞机正在起飞。”），5.000分;
  * "A woman is eating something."（“一个女人在吃东西。”），"A woman is eating meat."（“一个女人在吃肉。”），3.000分;
  * "A woman is dancing."（一个女人在跳舞。），"A man is talking."（“一个人在说话。”），0.000分。
<center><img src='../img/bert-two-seqs.svg'></center>
<center>图15.6.2 文本对分类或回归应用的BERT微调，如自然语言推断和语义文本相似性（假设输入文本对分别有两个词元和三个词元）</center><br>
- 与图15.6.1中的单文本分类相比，图15.6.2中的文本对分类的BERT微调在输入表示上有所不同。
- 对于文本对回归任务（如语义文本相似性），可以应用细微的更改，例如输出连续的标签值和使用均方损失：它们在回归中很常见。

- **要点：**
  - **文本对分类**: 以两段文本作为输入，分类任务的输出是它们之间**关系的类别**。
  - **文本对回归**: 输入为文本对，任务输出一个连续值，表示文本间的**语义相似度**。
  - **语义文本相似度**: 一种文本对回归任务，用分数（0-5）来评估两个句子的语义相似性。
  - **模型结构**: BERT模型微调用于处理输入的文本对，"\<cls\>"标记用于编码两个文本合并后的整体信息。
  - **损失函数**: 文本对回归任务可以使用均方损失来预测连续的相似度分数。

## 15.6.3 BERT文本标注
- 现在让我们考虑词元级任务，比如**文本标注（text tagging）**，其中**每个词元都被分配了一个标签**。
- 在文本标注任务中，**词性标注**为每个单词分配**词性标记**（例如，形容词和限定词）。
- 根据单词在句子中的作用。
  - 如在Penn树库II标注集中，句子“John Smith‘s car is new”应该被标记为“NNP（名词，专有单数）NNP POS（所有格结尾）NN（名词，单数或质量）VB（动词，基本形式）JJ（形容词）”。
<center><img src='../img/bert-tagging.svg'></center>
<center>图15.6.3 文本标记应用的BERT微调，如词性标记。假设输入的单个文本有六个词元。</center>
- 图15.6.3中说明了文本标记应用的BERT微调。
- 与图15.6.1相比，唯一的区别在于，在文本标注中，输入文本的**每个词元**的BERT表示被送到相同的额外全连接层中，以输出词元的标签，例如**词性标签**。

- **要点：**
  - 文本标记是一种词元级任务，每个词元被分配一个标签。
  - 词性标注是文本标记的一种形式，为每个单词分配词性（如形容词、限定词）。
  - 文本标记根据单词在句子中的语法作用进行。
  - 例如，根据Penn树库II标注集，"John Smith's car is new" 标记为 "NNP NNP POS NN VB JJ"。
  - 文本标记可以通过对BERT模型进行微调来实现，每个词元的表示通过额外的全连接层，以获取其标签。

## 15.6.4 BERT问答
- 作为另一个词元级应用，**问答**反映阅读理解能力。
- 例如，斯坦福问答数据集（Stanford Question Answering Dataset，SQuAD v1.1）**由阅读段落和问题组成，其中每个问题的答案只是段落中的一段文本（文本片段）**  。
- 举个例子,考虑一段话：
  - “Some experts report that a mask's efficacy is inconclusive.However,mask makers insist that their products,such as N95 respirator masks,can guard against the virus.”（“一些专家报告说面罩的功效是不确定的。然而，口罩制造商坚持他们的产品，如N95口罩，可以预防病毒。”）还有一个问题“Who say that N95 respirator masks can guard against the virus?”（“谁说N95口罩可以预防病毒？”）。
  - 答案应该是文章中的文本片段“mask makers”（“口罩制造商”）。
- 因此，**SQuAD v1.1的目标是在给定问题和段落的情况下预测段落中文本片段的开始和结束。**
<center><img src='../img/bert-qa.svg'></center>
<center>图15.6.4 对问答进行BERT微调（假设输入文本对分别有两个和三个词元）</center><br>
- 为了微调BERT进行问答，在BERT的输入中，**将问题和段落分别作为第一个和第二个文本序列**。
- 为了预测文本片段开始的位置，相同的额外的全连接层将把来自位置$i$的任何词元的BERT表示转换成标量分数$s_i$。
- 文章中所有词元的分数还通过softmax转换成概率分布，从而为文章中的每个词元位置$i$分配作为文本片段开始的概率$p_i$。
- 预测文本片段的结束与上面相同，只是其额外的全连接层中的参数与用于预测开始位置的参数无关。
- 当预测结束时，位置$i$的词元由相同的全连接层变换成标量分数$e_i$。 
- 图15.6.4描述了用于问答的微调BERT。
- 对于问答，监督学习的训练目标就像最大化真实值的开始和结束位置的对数似然一样简单。
  - 当预测片段时，我们可以计算从位置$i$到位置$j$的有效片段的分数$s_i + e_j$（$i \leq j$），并输出分数最高的跨度。

- **要点：**
  -  问答任务是反映阅读理解的词元级应用。
  - SQuAD v1.1数据集包括阅读段落和问题，答案是段落中的一个文本片段。
  - 目标是预测给定问题的答案在段落中的**开始**和**结束**位置。
  - 微调BERT时，问题和段落分别作为输入文本的两个序列。
  - 一个全连接层把BERT的每个词元表示转换成开始位置的分数，这些分数经softmax转换成概率分布。
  - 另一个独立的全连接层用于转换结束位置的分数，并进行相同的概率分布转换。
  - 训练包括最大化实际答案开始和结束位置的对数似然。
  - 预测答案文本片段通过计算和指定开始和结束位置的分数组合，并选择**分数最高的跨度**。

## 小结

* 对于序列级和词元级自然语言处理应用，BERT只需要最小的架构改变（额外的全连接层），如单个文本分类（例如，情感分析和测试语言可接受性）、文本对分类或回归（例如，自然语言推断和语义文本相似性）、文本标记（例如，词性标记）和问答。
* 在下游应用的监督学习期间，额外层的参数是从零开始学习的，而预训练BERT模型中的所有参数都是微调的。

------------
- **说明：SQuAD数据集**
  - SQuAD（Stanford Question Answering Dataset）1.1是自然语言处理（NLP）领域广泛使用的机器阅读理解基准数据集，其数据结构设计紧密贴合真实场景中的问答任务需求。

  - **（1）数据集整体架构**。SQuAD1.1 采用 **JSON** 格式组织数据，整体分为三级嵌套结构：
    ```python
    {  
        "version": "1.1",  # 数据集版本标识
        "data": [          # 文档集合（核心数据）
            {
                "title": "文档标题",
                "paragraphs": [
                    {
                        "context": "文本段落",
                        "qas": [  # 问题-答案对
                            {
                                "id": "唯一标识符",
                                "question": "问题文本",
                                "answers": [
                                    {
                                        "text": "答案片段",
                                        "answer_start": 起始字符位置
                                    }
                               ],
                              "is_impossible": False  # 标记是否无答案（SQuAD2.0新增）
                            }
                        ]
                    }
                ]
            }
        ]
    }
    ```

  - **（2）核心字段详解**
    - **文档级（data[]）**
    | 字段        | 类型     | 描述                                                                 |
    |-------------|----------|----------------------------------------------------------------------|
    | `title`     | string   | 文档主题（如"Super Bowl 50"），用于提供上下文背景                     |
    | `paragraphs`| array    | 文档分割后的文本段落（平均每段落约4-5句话）                           |

    - **段落级（paragraphs[]）**
    | 字段        | 类型     | 描述                                                                 |
    |-------------|----------|----------------------------------------------------------------------|
    | `context`   | string   | 原始文本段落（平均长度约300字符）                                    |
    | `qas`       | array    | 基于该段落生成的问题-答案对（平均每段落5个问题）                      |

    - **问答对级（qas[]）**
    | 字段            | 类型     | 描述                                                                 |
    |-----------------|----------|----------------------------------------------------------------------|
    | `id`            | string   | 唯一ID（格式如"56be4db0acb8001400a502ec"）                          |
    | `question`      | string   | 人工标注的问题（如"What team won Super Bowl 50?"）                  |
    | `answers`       | array    | 正确答案列表（SQuAD1.1中每个问题仅有1个标准答案）                    |
    | → `text`        | string   | 答案文本片段（必须直接出自`context`）                                |
    | → `answer_start`| integer  | 答案在`context`中的起始字符位置（从0开始计数）                       |



------


- **附录：BERT问答系统实例**

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertModel
from torch.optim import AdamW
import json
from pathlib import Path
from urllib.request import urlretrieve
from tqdm import tqdm

- SQuAD数据集下载地址：
  - 训练数据集: https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
  - 开发数据集: https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json

In [2]:
# SQuAD数据加载
def read_squad(path):
    with open(path, 'r', encoding='utf-8') as f:
        squad = json.load(f)
    
    examples = []
    for article in squad['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                if qa['answers']:
                    answer = qa['answers'][0]
                    examples.append({
                        'context': context,
                        'question': question,
                        'answers': answer
                    })
    return examples

- 数据预处理类

In [3]:
# 数据预处理类 
class SquadDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=384):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        example = self.data[idx]
        question = example['question']
        context = example['context']
        answer = example['answers']
        
        encoding = self.tokenizer(
            question, context,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            return_offsets_mapping=True
        )
        
        input_ids = encoding['input_ids'].squeeze()
        offset_mapping = encoding['offset_mapping'].squeeze()
        
        start_char = answer['answer_start']
        end_char = start_char + len(answer['text'])
        
        start_positions = 0
        end_positions = 0
        for i, (start, end) in enumerate(offset_mapping):
            if start <= start_char < end:
                start_positions = i
            if start < end_char <= end:
                end_positions = i
                break
        
        return {
            'input_ids': input_ids,
            'attention_mask': encoding['attention_mask'].squeeze(),
            'start_positions': torch.tensor(start_positions),
            'end_positions': torch.tensor(end_positions)
        }

- BERT问答模型
  - 此处使用bert-base-uncased，即不区分大小写的英文预训练BERT模型。
    - 下载地址：https://huggingface.co/google-bert/bert-base-uncased/tree/main
  - 可尝试使用bert-base-chinese预训练BERT模型进行中文问题问答。
    - 下载地址：https://huggingface.co/google-bert/bert-base-chinese/tree/main 

In [4]:
# BERT问答模型 
class BertForQA(nn.Module):
    # 使用本地实现下载的BERT预训练模型，基本小写版
    def __init__(self, bert_model=r'../weights/bert-base-uncased'):
        super().__init__()
        # 从指定路径加载预训练的BERT模型（若本地不存在则自动下载）
        self.bert = BertModel.from_pretrained(bert_model)
        # 参数'2'对应答案的起始位置和结束位置两个输出
        self.qa_outputs = nn.Linear(self.bert.config.hidden_size, 2)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        # 最后一层所有token的隐藏状态（形状：[batch_size, seq_length, hidden_size]）
        sequence_output = outputs.last_hidden_state
        # 将每个token的隐藏状态通过全连接层，得到每个token作为答案开始/结束位置的分数
        # 输出形状：[batch_size, seq_length, 2]
        logits = self.qa_outputs(sequence_output)
        # 沿最后一个维度（dim=-1）分割logits，分别得到start和end的logits
        # 输出形状：两个[batch_size, seq_length, 1]的张量
        start_logits, end_logits = logits.split(1, dim=-1)
        return start_logits.squeeze(-1), end_logits.squeeze(-1)

- 训练函数

In [5]:
# 训练函数
def train_bert_qa(model, train_loader, device, epochs=3, lr=5e-5):
    model.train()
    model.to(device)
    # BERT专用优化器，解决权重衰减问题
    optimizer = AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_pos = batch['start_positions'].to(device)
            end_pos = batch['end_positions'].to(device)
            
            optimizer.zero_grad()
            # 每个token作为答案起点和终点的分数（两个张量的形状皆是：[batch_size, seq_len]）
            start_logits, end_logits = model(input_ids, attention_mask)
            loss = criterion(start_logits, start_pos) + criterion(end_logits, end_pos)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')

- 预测函数

In [6]:
# 预测函数
def predict_qa(model, context, question, tokenizer, device):
    model.eval()
    encoding = tokenizer(
        question, context,
        max_length=384,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    
    with torch.no_grad():
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)
        start_logits, end_logits = model(input_ids, attention_mask)
        
        start_idx = torch.argmax(start_logits)
        end_idx = torch.argmax(end_logits)
        
        all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        answer_tokens = all_tokens[start_idx: end_idx+1]
        answer = tokenizer.convert_tokens_to_string(answer_tokens)
    
    return answer

- 运行模型

In [7]:
# 运行模型
data_dir='../data/squad'
train_data = read_squad(Path(data_dir) / 'train-v1.1.json')
dev_data = read_squad(Path(data_dir) / 'dev-v1.1.json') #开发数据集可用于F1指标测量
    
# 初始化模型和tokenizer
#tokenizer = BertTokenizer.from_pretrained(r'../weights/bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained(r'../weights/bert-base-uncased')

model = BertForQA()
    
# 创建数据集
train_dataset = SquadDataset(train_data[:1000], tokenizer)  # 使用部分数据示例
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    
# 训练
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_bert_qa(model, train_loader, device, epochs=3)


Epoch 1: 100%|██████████████████████████████████████████████████████████████████| 125/125 [00:15<00:00,  8.13it/s]


Epoch 1, Loss: 8.1354


Epoch 2: 100%|██████████████████████████████████████████████████████████████████| 125/125 [00:15<00:00,  8.16it/s]


Epoch 2, Loss: 4.8147


Epoch 3: 100%|██████████████████████████████████████████████████████████████████| 125/125 [00:15<00:00,  8.17it/s]

Epoch 3, Loss: 2.5525


- 测试1：
  - 段落："量子计算公司QuantTech于2023年发布了全球首款室温超导量子处理器'Phoenix'，该处理器能在25摄氏度环境下稳定运行，突破了传统量子计算机需要接近绝对零度的限制。Phoenix采用新型碳基超导材料，单量子比特相干时间达到1毫秒。"
  - 问题："QuantTech发布的量子处理器在什么温度下可以稳定运行？"
  - 答案："25摄氏度"

In [16]:
context = "QuantTech, a quantum computing company, \
           unveiled the world's first room-temperature superconducting quantum processor 'Phoenix' in 2023.\
           This processor can operate stably at 25°C,\
           breaking the traditional requirement of near-absolute-zero conditions.\
           Phoenix utilizes novel carbon-based superconducting materials, \
           achieving a single-qubit coherence time of 1 millisecond."
question = "At what temperature can QuantTech's quantum processor operate stably?"
    
answer = predict_qa(model, context, question, tokenizer, device)
print(f"\nQuestion: {question}")
print(f"Answer: {answer}")


Question: At what temperature can QuantTech's quantum processor operate stably?
Answer: 25°c


- 测试2：
  - 段落："丝绸之路上的古代城市撒马尔罕曾是帖木儿帝国的首都。考古发现表明，该城在14世纪拥有世界上最大的天文观测台'乌鲁格别克天文台'，其制作的星表精度保持了200年的世界纪录。"
  - 问题："帖木儿帝国的首都叫什么名字？"
  - 答案： "撒马尔罕"


In [9]:
context = "The ancient city of Samarkand along the Silk Road served as the capital of the Timurid Empire.\
            Archaeological findings reveal it housed the world's largest astronomical observatory, \
            the 'Ulugh Beg Observatory', in the 14th century, \
            whose star catalog maintained world-record precision for 200 years."
question = "What was the capital city of the Timurid Empire?"
    
answer = predict_qa(model, context, question, tokenizer, device)
print(f"\nQuestion: {question}")
print(f"Answer: {answer}")


Question: What was the capital city of the Timurid Empire?
Answer: samarkand


- 测试3：
  - 段落："深海热泉区发现的'鳞脚蜗牛'（Chrysomallon squamiferum）是已知唯一一种外壳含铁的软体动物。其外壳由硫化铁层构成，能承受250个大气压，这种生物为新型抗压材料研发提供了灵感。"
  - 问题："哪种海洋生物的外壳含有硫化铁？"
  - 答案："鳞脚蜗牛"
- 本测试案例错误，正确答案应该是："scaly-foot gastropod"

In [20]:
context = "The 'scaly-foot gastropod' (Chrysomallon squamiferum) \
          discovered near deep-sea hydrothermal vents is the only known mollusk \
          with iron-containing shells. Its armor consists of iron sulfide layers \
          that can withstand 250 atmospheres of pressure, \
          inspiring new pressure-resistant material designs."
question = "Which marine creature has iron sulfide shells?"
answer = predict_qa(model, context, question, tokenizer, device)
print(f"\nQuestion: {question}")
print(f"Answer: {answer}")


Question: Which marine creature has iron sulfide shells?
Answer: armor consists of iron sulfide layers


--------------